<a href="https://colab.research.google.com/github/AbrahamKong/CMPE257-NLP_AlternusVera/blob/master/Merging_models_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Streamlite App of all models**

In [ ]:
! pip install streamlit
! pip install transformers
! pip install spacy --upgrade
! python -m spacy download en_core_web_md
! python -m spacy download en_core_web_sm
! pip install swifter
! pip install spacy-universal-sentence-encoder
!pip install dgl==0.6.0
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

In [ ]:
! streamlit run final_app.py & npx localtunnel --port 8501

## **All Model**

In [ ]:
%%writefile final_app.py
import streamlit as st
import requests
import json
import csv
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
from string import punctuation
import seaborn as sns
import re
import nltk
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
import matplotlib.pyplot as plt
from scipy import sparse
import warnings
warnings.filterwarnings("ignore")
from numba import jit
import pandas as pd
import numpy as np
import dgl
from dgl.dataloading import GraphDataLoader
from dgl.data import DGLDataset
from dgl.nn.pytorch import GraphConv, GATConv, GatedGraphConv, DotGatConv
from dgl.nn import AvgPooling, MaxPooling
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss
import torch.nn as nn
import operator
import scipy.sparse as sp
import spacy
import joblib
import torch.nn.functional as F
import spacy_universal_sentence_encoder
from transformers import pipeline


avinash_model = joblib.load("/content/liar_final.sav")
# abraham_model = joblib.load("/content/abraham_final.sav")


medium_nlp = spacy_universal_sentence_encoder.load_model('en_use_md')
nlp = spacy.load("en_core_web_sm")

def load_models(model, folds, avinash_model):
    model_list = []
    
    for i in range(folds):
        # print('Loading weights')
        model.load_state_dict(torch.load(f'/content/drive/My Drive/ML-Spring-2022/Data Miners/Assignments/Pooja/Assignment - 12/Graph_model/fold-{i}.pt'))
        model.eval()
        model_list.append(model)
        model_list.append(avinash_model)
        #model_list.append(abraham_model)        
    return model_list

def cognitive_complexisty(text):
  words=[x for x in text.split(' ')]
  char_count_per_word= [len(x) for x in text.split(' ')]
  Average_character_per_word=sum(char_count_per_word)/len(char_count_per_word)
  unique_word_ratio = len(set(words))/len(words)
  return (Average_character_per_word,unique_word_ratio)

def text_cleaning(text):
    '''
    Make text lowercase, remove text in square brackets,remove links,remove special characters
    and remove words containing numbers.
    '''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) # remove special chars
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    
    return text

def consolidated_micro_factors(dataframe):
  dataframe['cleaned_text']=dataframe['news'].apply(text_cleaning)
  dataframe.drop(columns=['news'],inplace=True)
  dataframe['cognitive_complexisty']=dataframe['cleaned_text'].apply(cognitive_complexisty)
  Average_character_per_word,unique_word_ratio = zip(*dataframe['cognitive_complexisty'].values)
  dataframe['Average_character_per_word']=Average_character_per_word
  dataframe['unique_word_ratio'] = unique_word_ratio
  dataframe.drop(columns=['cognitive_complexisty'],inplace=True)
  dataframe['Verbal_immediacy']=dataframe['cleaned_text'].apply(findPronoun_Verb)
  ratio_of_past_tense_verb, ratio_of_PRON_first_person,ratio_of_PRON_third_person = zip(*dataframe['Verbal_immediacy'].values)
  dataframe['ratio_of_past_tense_verb'] = ratio_of_past_tense_verb
  dataframe['ratio_of_PRON_first_person'] = ratio_of_PRON_first_person
  dataframe['ratio_of_PRON_third_person'] = ratio_of_PRON_third_person
  dataframe.drop(columns=['Verbal_immediacy'],inplace=True)
  @jit
  def test(x):
    return classifier1(x)

  #emotion1 = dataframe['cleaned_text'].swifter.apply(test)
  emotion1 = dataframe['cleaned_text'].apply(test)
  
  anger=[]
  disgust=[]
  fear=[]
  joy=[]
  neutral=[]
  sadness=[]
  surprise=[]
  
  for x in range(len(emotion1)):
    # print(emotion1[x][0][0]['score'])
    anger.append(emotion1[x][0][0]['score'])
    disgust.append(emotion1[x][0][1]['score'])
    fear.append(emotion1[x][0][2]['score'])
    joy.append(emotion1[x][0][3]['score'])
    neutral.append(emotion1[x][0][4]['score'])
    sadness.append(emotion1[x][0][5]['score'])
    surprise.append(emotion1[x][0][6]['score'])
  emotions_scores1_df = pd.DataFrame(data=[anger,disgust,fear,joy,neutral,sadness,surprise]).T.rename(columns={0:'anger',1:'disgust',2:'fear',3:'joy',4:'neutral',5:'sadness',6:'surprise'})
  dataframe = pd.merge(dataframe,emotions_scores1_df,left_index=True,right_index=True)
 # dataframe['word_vectors']=dataframe['cleaned_text'].swifter.apply(lambda x : list(medium_nlp(x).vector))
  dataframe['word_vectors']=dataframe['cleaned_text'].apply(lambda x : list(medium_nlp(x).vector))
  dataframe.drop(columns=['cleaned_text'],inplace=True)
  X=dataframe.iloc[:,1:-1]
  X_word_vector = pd.DataFrame(list(dataframe['word_vectors']))
  dataframe_final = pd.merge(X,X_word_vector,left_index=True,right_index=True)
  loaded_model = joblib.load("/content/deception_final.sav")
  loaded_model1 = joblib.load("/content/fakedit_final.sav")
 # final_model= joblib.load("/content/liar_final.sav")
  deception=loaded_model.predict(dataframe_final)
  dataframe_final['deception']=deception
  fakedit_label=loaded_model1.predict(dataframe_final)
  dataframe_final['fakedit_label']=np.argmax(fakedit_label,axis=1)
  #prediction =  np.argmax(final_model.predict(dataframe_final),axis=1)
  return dataframe_final

def findPronoun_Verb(text):
    PRON_first_person = 0
    PRON_third_person = 0
    pron_count=0
    past_tens_verb_count=0
    verb_count=0
    doc = nlp(text)
    for token in doc:
          if token.pos_=='PRON':
            pron_count+=1
            if token.morph.get("Person") == ["1"] and token.morph.get("Number") == ['Sing']:
              PRON_first_person+=1
            if token.morph.get("Number")==["3"]:
              PRON_third_person+=1
          if token.pos_=='VERB':
            verb_count+=1
            if token.morph.get("Tense")==['Past']:
              past_tens_verb_count+=1

    ratio_of_past_tense_verb = past_tens_verb_count/verb_count if verb_count else 0
    ratio_of_PRON_first_person = PRON_first_person/pron_count if pron_count else 0
    ratio_of_PRON_third_person = PRON_third_person/pron_count if pron_count else 0
    return ratio_of_PRON_first_person,ratio_of_PRON_third_person,ratio_of_past_tense_verb

def get_final_AModel(dataframe_final):
  final_model= joblib.load("/content/liar_final.sav")
  #prediction =  np.argmax(final_model.predict(dataframe_final),axis=1)
  logits =  final_model.predict(dataframe_final)
  return logits

classifier1 = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True, truncation='only_first')

class args:
    max_epochs = 10
    lr = 1e-2
    batch_size = 32
    embedding_dim = 50
    hidden_dim = 50
    num_heads = 8   # used for attention model
    n_folds = 5
    window_size = 3


def test(args, n_classes, test_data):
  MAX_TRUNC_LEN = 128

  word_embeddings_dim = args.embedding_dim

  shuffle_doc_words_list = list(test_data['news'].values)

  word_set = set()

  # Returns list of tuples. The first element of each tuple specifies the word present in the description but not in the embeddings  and the second element specifies the count of that word in the descriptions. The tuples are sorted in the descending order of their count.
  def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    # print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    # print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

  # Returns dictionary with keys as words in the sentences and values as their count of occurence
  def build_vocab(sentences, verbose =  True):
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

  # Returns list of adjacency matrix and list of node matrix
  def build_graph(test_data, start, end, truncate = False, weighted_graph = True):
    x_adj = []
    x_feature = []
    doc_len_list = []
    vocab_set = set()
    # print("vocab", vocab)

    for i in tqdm(range(start, end)):

        doc_words = list(test_data['news'].values)[i].split()
        if truncate:
            doc_words = doc_words[:128]
        doc_len = len(doc_words)

        doc_vocab = list(set(doc_words))
        doc_nodes = len(doc_vocab)

        doc_len_list.append(doc_nodes)
        vocab_set.update(doc_vocab)

        doc_word_id_map = {}
        for j in range(doc_nodes):
            doc_word_id_map[doc_vocab[j]] = j

        # sliding windows
        windows = []
        if doc_len <= 3:
            windows.append(doc_words)
        else:
            for j in range(doc_len - 3 + 1):
                window = doc_words[j: j + 3]
                windows.append(window)

        word_pair_count = {}
        for window in windows:
            for p in range(1, len(window)):
                for q in range(0, p):
                    word_p = window[p]
                    word_p_id = word_id_map[word_p]
                    word_q = window[q]
                    word_q_id = word_id_map[word_q]
                    if word_p_id == word_q_id:
                        continue
                    word_pair_key = (word_p_id, word_q_id)
                    # word co-occurrences as weights
                    if word_pair_key in word_pair_count:
                        word_pair_count[word_pair_key] += 1.
                    else:
                        word_pair_count[word_pair_key] = 1.
                    # bi-direction
                    word_pair_key = (word_q_id, word_p_id)
                    if word_pair_key in word_pair_count:
                        word_pair_count[word_pair_key] += 1.
                    else:
                        word_pair_count[word_pair_key] = 1.
    
        row = []
        col = []
        weight = []
        features = []
        for key in word_pair_count:
            p = key[0]
            q = key[1]
            row.append(doc_word_id_map[vocab[p]])
            col.append(doc_word_id_map[vocab[q]])
            weight.append(word_pair_count[key] if weighted_graph else 1.)
        adj = sp.csr_matrix((weight, (row, col)), shape=(doc_nodes, doc_nodes))
    
        for k, v in sorted(doc_word_id_map.items(), key=lambda x: x[1]):
            features.append(word_embeddings[k] if k in word_embeddings else oov[k])

        x_adj.append(adj)
        x_feature.append(features)

    
    return x_adj, x_feature

  # Get graph and app target label
  class GraphDataset(DGLDataset):
    def __init__(self, x_adj, x_feature, targets = None):
        self.adj_matrix = x_adj
        self.node_matrix = x_feature
        self.targets = targets
        
    def __len__(self):
        return len(self.adj_matrix)
    
    def __getitem__(self, idx):
        scipy_adj = self.adj_matrix[idx]
        G = dgl.from_scipy(scipy_adj)
        G.ndata['feat'] = torch.stack([torch.tensor(x, dtype = torch.float) for x in self.node_matrix[idx]])
        
        
        if self.targets is not None:
            label = self.targets[idx]
            
            return G, torch.tensor(label, dtype = torch.long)
        
        return G

  # Graph Neural Network with Attention Layers where a dot product is performed between node features  
  class GATDotClassifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, num_heads, n_classes):
        super(GATDotClassifier, self).__init__()
        self.hid_dim = hidden_dim
        self.gat1 = DotGatConv(in_dim, hidden_dim, num_heads, allow_zero_in_degree=True)
        self.gat2 = DotGatConv(hidden_dim*num_heads, hidden_dim, 1, allow_zero_in_degree=True)
        self.avgpooling = AvgPooling()
        self.classify = nn.Linear(hidden_dim, n_classes)

    def forward(self, g, h):
        # Apply graph convolution and activation.
        bs = h.shape[0]
        h = F.relu(self.gat1(g, h))
        h = h.reshape(bs, -1)
        h = F.relu(self.gat2(g, h))
        h = h.reshape(bs, -1)
        h = self.avgpooling(g, h)
        
        return self.classify(h)

  for doc_words in shuffle_doc_words_list:
      words = doc_words.split()
      word_set.update(words)
      
  vocab = list(word_set)
  vocab_size = len(vocab)

  word_id_map = {}
  for i in range(vocab_size):
      word_id_map[vocab[i]] = i
      
  oov = {}
  for v in vocab:
      oov[v] = np.random.uniform(-0.1, 0.1, word_embeddings_dim)
      
  GLOVE_EMBEDDING_PATH = '/content/glove.6B.50d.txt'

  word_embeddings = {}

  with open(GLOVE_EMBEDDING_PATH, 'r') as f:
    for line in f.readlines():
      data = line.split()
      word_embeddings[str(data[0])] = list(map(float,data[1:]))

  vocab = build_vocab(list(test_data['news'].apply(lambda x:x.split())))

  oov = check_coverage(vocab,word_embeddings)
  oov[:10]
  window_size = 3
  #labels = test_data['label'].unique()
  #label2idx = {l:i for i,l in enumerate(sorted(labels))}
  #idx2label = {v:k for k,v in label2idx.items()}

  # test_data['label'] = test_data['label'].apply(lambda x: label2idx[x])
  num_classes = n_classes
  window_size = 3
  x_adj, x_feature = build_graph(test_data, start=0, end=len(test_data), weighted_graph = True)
  
  testdataset = GraphDataset(x_adj, x_feature)
  testloader = GraphDataLoader(testdataset, batch_size = args.batch_size, shuffle = False)
  
  model = GATDotClassifier(args.embedding_dim, args.hidden_dim, args.num_heads, num_classes)
  model_list = load_models(model, args.n_folds, avinash_model)
  
  pred_list = []
  pred_softmax = []

  # print(vocab)

  with torch.no_grad():
      for idx, G in enumerate(tqdm(testloader)):
          h = G.ndata['feat'].float()
          logits = 0
          for mod in model_list:
            if mod == avinash_model:
              intent = test_data.rename(columns = {'news':'statement'})
              final_dataset = consolidated_micro_factors(intent.copy())
              logits =  get_final_AModel(final_dataset)
            else:
              log = mod(G, h)
              # blending of logits from all 5 models. This helps in getting more robust predictions. 
              logits_numpy = (log.softmax(-1) / (args.n_folds)).numpy()
              #logits += log.softmax(-1) / (args.n_folds)

      logits_avg = np.mean((logits, logits_numpy), axis = 0)
      # print(logits_avg)

      result = logits_avg.argmax(axis = 1)
  return result

def fetch_news():
  url = ('https://newsapi.org/v2/top-headlines?'
       'country=us&'
       'apiKey=78f8b5ac018e4c568badb6877ac4f778')
  response = requests.get(url)
  news_data = response.json()
  json_object = json.dumps(news_data)
  with open("news.json", "w") as outfile:
    outfile.write(json_object) 
  df = pd.read_json('/content/news.json')
  news_data = pd.json_normalize(news_data,record_path=['articles']).iloc[:10,:]
  return news_data

st.header("Click the below button to extract 10 news from Newsapi")
if st.button('Fetch News'):
  news = fetch_news()
  st.dataframe(news)
  st.markdown("Perform Prediction for each news")
  for index, row1 in news.iterrows():
    st.header(f"News - {index}")
    st.subheader('title')
    st.write(row1['title'])
    st.subheader("News Description")
    st.write(row1['description'])
    st.subheader("News Content")
    st.write(row1['content'])
    st.subheader("Source")
    st.write(row1['source.name'])
    st.subheader("Prediction")
    if row1['content'] ==None:
      if row1['description']==None:
        prediction = test(args, 6, pd.DataFrame({'news': row1['title']},index=[0]))
      else:
        prediction = test(args, 6, pd.DataFrame({'news': row1['description']},index=[0])) 
    else:
      prediction = test(args, 6, pd.DataFrame({'news': row1['content']},index=[0]))

    labels_dict = {'true' : 0, 'mostly-true': 1, 'half-true': 2, 'barely-true': 3, 'false': 4, 'pants-fire': 5}
    finding_category={y: x for x, y in labels_dict.items()}
    if prediction == 0:
      st.write('true')
    elif prediction ==1:
      st.write('mostly-true')
    elif prediction ==2:
      st.write('half-true')
    elif prediction ==3:
      st.write('barely-true')
    elif prediction ==4:
      st.write('false')
    else:
      st.write('pants-fire')    

Overwriting final_app.py


In [ ]:
! streamlit run final_app.py & npx localtunnel --port 8501

2022-05-13 18:11:32.882 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.849s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.199.60.83:8501

your url is: https://great-books-warn-35-199-60-83.loca.lt
  Stopping...
^C


## **Abraham_Model**

In [ ]:
politifact_df = pd.read_csv('/content/politifact_abraham.csv')

In [ ]:
politifact_df.head(10)

In [ ]:
def cleaning(raw_news):
    # import nltk
    # nltk.download('punkt')
    # nltk.download('wordnet')
    
    # 1. Remove non-letters/Special Characters and Punctuations
    news = re.sub("[^a-zA-Z]", " ", raw_news)
    
    # 2. Convert to lower case.
    news =  news.lower()
    
    # 3. Tokenize.
    news_words = nltk.word_tokenize( news)
    
    # 4. Convert the stopwords list to "set" data type.
    stops = set(nltk.corpus.stopwords.words("english"))
    
    # 5. Remove stop words. 
    words = [w for w in  news_words  if not w in stops]
    
    # 6. Lemmentize 
    wordnet_lem = [ WordNetLemmatizer().lemmatize(w) for w in words ]
    
    # 7. Stemming
    stems = [nltk.stem.SnowballStemmer('english').stem(w) for w in wordnet_lem ]
    
    # 8. Join the stemmed words back into one string separated by space, and return the result.
    return " ".join(stems)

In [ ]:
# Reference https://gist.github.com/AhmedHani/81bd40ff3ce4d9131f02c89fe099d490
def get_sentences(text):
    return re.split('\? |, |!|\n', text)

def get_words(sentences):
    return sum([sentence.split() for sentence in sentences], [])

def get_average_word_length(words):
    return sum([len(word) for word in words]) // len(words)
  
def get_average_word_based_sentence_length(sentences):
    return sum([len(sentence.split()) for sentence in sentences]) // len(sentences)

def get_average_chars_based_sentence_length(sentences):
    return sum([len(sentence) for sentence in sentences]) // len(sentences)
  
def get_punctuations_count(text):
    return len([c for c in text if c in string.punctuation])

def vocab_richness_ratio(words):
  return float(len(set(words))) / float(len(words))

def shannon_entropy(words):
  import math
  import numpy as np
  import scipy as sc
  from collections import Counter

  length = len(words)
  freqs = Counter(words)
  distribution = np.array(list(freqs.values()))
  distribution = np.true_divide(distribution, length)
    
  E = sc.stats.entropy(distribution, base=2)

  return E

def syllables_count(words, vowels="aeiouy"):
  total_count = 0.0

  for word in words:
    word = word.lower()
    count = 0
  
    if word[0] in vowels:
      count += 1
  
    for i in range(1, len(word)):
      if word[i] in vowels and word[i - 1] not in vowels:
        count += 1

        if word.endswith("e"):
          count -= 1
    
    if count == 0:
      count = 1
    
    total_count += count

  return total_count, (total_count // len(words)) # Average

def flesch_reading_ease(words, sentences, avg_word_based_sentence_length):
  # 206.835 - (1.015 * Average Number of Words Per Sentence) - (84.6 * Average Syllables Count Per Sentence)
  _, avg_syllables_count = syllables_count(words)

  return 206.835 - (1.015 * avg_word_based_sentence_length) - (84.6 * avg_syllables_count)

In [ ]:
def mergeLabel(r):
        '''
        Merge the PolitiFacts's 8 labels to Liar Liar's 6 data label
        Change full-flop to pants-fire, and half-flip to barely-true
        '''

        # v = r['Label']

        if (r == 'half-flip'):
            r = 'pants-fire'
        if (r == 'full-flop'):
            r = 'barely-true'
        return r 


In [ ]:
def mergeNumberLabel(r):
        '''
        Convert labels to Integers
        '''

        # v = r['Label']

        if (r == 'true'):
            r = 0
        if (r == 'mostly-true'):
            r = 1
        if (r == 'half-true'):
            r = 2
        if (r == 'barely-true'):
            r = 3
        if (r == 'false'):
            r = 4
        if (r == 'pants-fire'):
            r = 5
        if (r == 'half-flip'):
            r = 5
        if (r == 'full-flop'):
            r = 3
        return r 

In [ ]:
politifact_df['Label'] = politifact_df["Label"].apply(mergeNumberLabel)

In [ ]:
politifact_df.head(10)

In [ ]:
def corpursStyleEval(df): 
  train_final = df.copy()

  train_final['label'] = train_final["Label"].apply(mergeNumberLabel)

  train_final['sentences'] = train_final['clean'].apply(get_sentences)
  # train_final['n_sentences'] = politifact_df["sentences"].apply(len)
  # print(f'number of sentences: {n_sentences}')
          
  train_final['words'] = train_final['sentences'].apply(get_words)
  # train_final['n_words'] = len(train_final['words'])
  # print(f'number of words: {n_words}')
          
  train_final['avg_word_length'] = train_final['words'].apply(get_average_word_length)
  # print(f'average word length: {avg_word_length}')
          
  train_final['avg_word_based_sentence_length'] = train_final['sentences'].apply(get_average_word_based_sentence_length)
  # print(f'average word-based sentence length: {avg_word_based_sentence_length}')
          
  train_final['avg_char_based_sentence_length'] = train_final['sentences'].apply(get_average_chars_based_sentence_length)
  # print(f'average char-based sentence length: {avg_char_based_sentence_length}')
          
  train_final['punc_count'] = train_final['clean'].apply(get_punctuations_count)
  # print(f'punctuations count: {punc_count}')

  train_final['richness_ratio'] = train_final['words'].apply(vocab_richness_ratio)

  train_final['flesch_score']=0
  for i in range(train_final.shape[0]):
    train_final['flesch_score'][i] = flesch_reading_ease(train_final['words'][i], train_final['sentences'][i], train_final['avg_word_based_sentence_length'][i])

  return train_final[['clean', 'label', 'richness_ratio', 'flesch_score']].copy()

In [ ]:
politifact_df['clean'] = politifact_df["Statement"].apply(cleaning)
politifact_df = corpursStyleEval(politifact_df)

In [ ]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import MinMaxScaler
import pickle

from tensorflow.keras.layers import *
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras import activations
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [ ]:
politifact_df.to_csv('/content/final_abraham.csv')

In [ ]:
politifact_df.head(10)

In [ ]:
train_final = politifact_df[politifact_df['flesch_score'].notna()]
X = train_final[['richness_ratio', 'flesch_score']].copy()
y = train_final['label']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

opt = Adam(learning_rate=0.01)
def BiLstm():
  model.add(layers.Dense(2, activation="relu"))
  model.add(layers.Dense(3, activation="relu"))
  model.add(Dense(6, activation=activations.softmax))
  model.build(input_shape = X_train.shape)
  model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = BiLstm()
model.summary()

In [ ]:
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test), 
                    epochs=200, 
                    batch_size=16, 
                   # callbacks=lrate, 
                    verbose=2)

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()